In [ ]:
 def display_summary(url):
        summary = summarize(url)
        display(Markdown(summary))  # Correct usage of Markdown

In [ ]:
import requests
import openai
import os
from bs4 import BeautifulSoup
from IPython.display import display, Markdown
from dotenv import load_dotenv  # Load environment variables
import json
import gradio as gr
from openai import OpenAI
# Load environment variables
load_dotenv()

# Initialize OpenAI API client with API key from environment variable
api_key = os.getenv("OPENAI_API_KEY")  # Use environment variable

# Headers for web scraping
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

system_prompt = "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."

# Website Class
class Website:
    def __init__(self, url):
        self.url = url
        self.title = "No title found"
        self.text = "No content available"
        try:
            print(f"Fetching website: {url}")
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"

            if soup.body:
                for irrelevant in soup.body(["script", "style", "img", "input"]):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")
            self.title = "Error"
            self.text = ""
        # Function to generate messages for OpenAI API
    def messages_for(website):
        message=self.text
        return [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": message}
        ]    

    # Function to call OpenAI API and summarize the webpage
    def chat_interface(url):
        website = Website(url)  # Create Website object
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=website.messages_for()
        )
        return response.choices[0].message.content  # Extract response content
   
gr.ChatInterface(fn=chat_interface, inputs="text", outputs="text").launch()